# Overview

In [3]:
import datetime
import json
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import typing

import seaborn as sns

MAX_STEPS = 25
MAX_CORES = 8

base_path = "/workspaces/muBench-experiment/gssi_experiment/gateway_offloading/results/large_experiment_1/"

pincirollis_data_path = "/workspaces/muBench-experiment/gssi_experiment/gateway_offloading/multi_N25.csv"
synth_df = pd.read_csv(pincirollis_data_path, header=0)
synth_df.describe()

,N_A,N_B,r_Z_A,r_Z_B,r_gw,r_A_s1,r_B_s2,r_B_s3,R0,R0_low,...,Us2,Us2_low,Us2_up,Us3,Us3_low,Us3_up,Ugw,Ugw_low,Ugw_up,sim_time_sec
0,0,25,0.01,0.01,1.250000e-01,0.083333,0.250000,0.142857,207.394334,204.485611,...,0.485361,0.473966,0.496755,0.852991,0.833860,0.872122,9.607230e-01,0.936403,9.850433e-01,12.413288
1,5,20,0.01,0.01,1.000000e+00,0.052632,0.090909,0.071429,234.464376,230.054291,...,0.771884,0.752784,0.790984,0.982965,0.967234,0.998696,1.063725e-01,0.104158,1.085865e-01,15.162418
2,4,21,0.01,0.01,2.500000e-01,0.062500,0.125000,0.090909,207.599198,204.659074,...,0.716570,0.695277,0.737862,0.987750,0.975201,1.000300,4.825850e-01,0.471229,4.939407e-01,15.169014
3,6,19,0.01,0.01,1.666667e-01,0.071429,0.166667,0.111111,177.424033,172.612817,...,0.601168,0.584595,0.617742,0.909018,0.888042,0.929994,8.418418e-01,0.821839,8.618448e-01,16.018047
4,0,25,0.01,0.01,1.000000e+15,0.050000,0.083333,0.066667,375.018656,365.662949,...,0.799797,0.780067,0.819526,0.996558,0.983861,1.009254,1.029954e-20,0.000000,2.614601e-12,17.442785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,24,1,0.01,0.01,2.000000e-01,0.066667,0.142857,0.100000,333.233053,327.845675,...,0.055936,0.054321,0.057552,0.079752,0.077446,0.082057,3.706845e-01,0.362208,3.791614e-01,8.389827
282,25,0,0.01,0.01,3.333333e-01,0.058824,0.111111,0.083333,425.721490,418.827660,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.755075e-01,0.170252,1.807631e-01,3.619553
283,25,0,0.01,0.01,2.500000e-01,0.062500,0.125000,0.090909,401.596761,393.651802,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.487802e-01,0.242188,2.553728e-01,3.315408
284,24,1,0.01,0.01,1.666667e-01,0.071429,0.166667,0.111111,315.343951,308.986374,...,0.048172,0.047146,0.049198,0.072222,0.070217,0.074226,4.745969e-01,0.463304,4.858899e-01,8.343186


# Dataframe creation

In [2]:
columns = [
    # Experiment Metadata
    "dashboard_intensity",
    "run_index",
    "replications",
    "trials",
    "cpu_cap",
    # Other metadata
    "experiment_start",
    "experiment_stop",
    # Results latency
    "real_avg_latency_ms",
    "synth_latency_ms",
    # Results CPU utilization.
    "cpu_utilization_readings",
]


# Results CPU utilization.
services = ["gw", "s1", "s2", "s3"]
real_cpu_key_format = "{service}_real_avg_cpu_utilization"
columns.extend([real_cpu_key_format.format(service=service) for service in services])
synth_cpu_key_format = "{service}_synth_cpu_utilization"
columns.extend([synth_cpu_key_format.format(service=service) for service in services])

# Request Delay

# CPU Utilization